# General data reader for AQUA 
## Streaming mode

The reader includes the ability to simulate a data streaming, that allows to retrieve chunks of data of specific time length.

In [1]:
from aqua import Reader

Let's load some OSI-SAF data. We first instantiate a `Reader` object specifying the type of data which we want to read from the catalogue. 
By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `startdate`, `enddate` and `aggregation`: 

`startdate` : is a string representing the starting date for streaming the data (e.g. '1979-01-01'). If it is not specified, the function will use the first date in the dataset.

`enddate`: is a string representing the ending date for streaming the data (e.g. '2020-12-31').  If it is not specified, the function will use the last date in the dataset.

`aggregation` (str): the streaming frequency.
 Available frequencies: 
- 1S, 5S etc ('S' stands for 'steps', if you want to keep the original data frequency. On hourly data, '5S' will return chunks of 5 hours each.)
  If no 'aggregation' is specified, '1S' will be the default streaming frequency.

- Panda's style: 1M, 7D etc. (keep in mind that the behavior of resampling in Pandas can be a bit counterintuitive when it comes to months (years) due to the variation in the length of months(years). It's safer to ask for multiple chunks of just hours or days.) 

- 'hourly', 'daily', 'weekly', 'monthly', 'yearly'.

- 'hour', 'day', 'pentad', 'week', 'month', 'year', 'decade'  (and their plural).
                                                                    
Keep in mind that if you want to stream chunks of a certain number of days, months etc. you must use the Panda's notation (e.g if you want to stream chunks of three months you should write `aggregation = '3M'`, `aggregation = '3months'` is not accepted).

Let's say we want to stream the data every three days. We need to call:

In [2]:
reader = Reader(model='MSWEP',exp='past',source='3hourly', streaming=True, aggregation = '6D')

Then the actual data are read with the `retrieve` method. By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `stream_step`, `stream_unit` and `stream_startdate`: 

`stream_unit` is a string representing the unit of time to stream the data by (e.g. 'hours', 'days', 'months') (default is None)

`stream_step` is an integer representing the number of time steps to stream the data by.  Default = 1

`stream_startdate` is a string representing the starting date for streaming the data (e.g. '1979-01-01'). If it is not specified, the function will use the first date in the dataset.

Let's say we want to stream the data every three days. We need to call:

In [3]:
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 45, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 1979-01-06T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(45, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

We correctly retrieved data from  1979-01-01 to 1979-01-06, keeping the original hourly temporal resolution. 

If `retrieve` is called a second time, it will return the subsequent chunk of 6 days data. This is because the method keeps track of the state of the streaming process through the use of internal attributes.

In [4]:
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 48, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-07 ... 1979-01-12T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(45, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

By calling `retrieve` a second time, we correctly retrieved data from 1979-01-07 to 1979-01-12.

If the retrieve method in streaming mode is called multiple times, it will return the data in chunks until all of the data has been streamed. The function will automatically determine the appropriate start and end points for each chunk based on the internal state of the streaming process.

If we want to reset the state of the streaming process we can call the `reset_stream` method. This means that if `retrieve` in streaming mode is called again after calling `reset_stream`, it will start streaming the input data from the beginning.

In [5]:
reader.reset_stream()
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 45, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 1979-01-06T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(45, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

We got back to the first three days, 1979-01-01 to 1979-01-06.

By default, the data stream will start from the first date of the input file.
If we want it to start from a specific date, we can use the parameter `startdate`.

In [6]:
reader = Reader(model='MSWEP',exp='past',source='3hourly', streaming=True, aggregation = '3D', startdate = '2019-05-01')
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 24, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 2019-05-01 ... 2019-05-03T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(24, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

Similarly, if we want to end the streaming at a specific date, before the natural end of the dataset, we can use the parameter `enddate`.

In [7]:
reader = Reader(model='MSWEP',exp='past',source='3hourly', streaming=True, aggregation = '3D', startdate = '2019-05-01', enddate = '2019-05-20')
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 24, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 2019-05-01 ... 2019-05-03T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(24, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

In [8]:
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 24, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 2019-05-04 ... 2019-05-06T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(12, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

As mentioned above, if we don't specify the `aggregation` parameter, the data is streamed keeping the the original time resolution of input data
(e.g. hour by hour for hourly data)
If we want chunks of a certain number of steps, we need to specify it. (e.g. aggregations = '6S')


In [9]:
reader = Reader(model='MSWEP',exp='past',source='3hourly', streaming=True, aggregation = '6S')
data = reader.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 6, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 1979-01-02
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Data variables:
    mtpr     (time, lat, lon) float32 dask.array<chunksize=(6, 105, 225), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed May 24 18:20:30 2023: cdo select,startdate=1979-01-01,e...

As requested, we got a six-hours chunk.

Another possibility to deal with data streaming is to call the `generator` method of the `Streaming` class.
This can be done in the Reader initialization through the argument `stream_generator = True`.

In [10]:
from aqua import Reader

In [11]:
reader = Reader(model='MSWEP',exp='past',source='3hourly', stream_generator = 'True', aggregation = 'monthly', startdate = '2019-01-01')
data_gen = reader.retrieve()
data_gen

<generator object Streaming.generator at 0x7f8788bc4b80>

`data_gen` is now a generator object that yields the requested one-month-long chunks of data. We can do operations with them by iterating on the generator object:

In [12]:
for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

start_date: 2019-01-01T00:00:00.000000000 stop_date: 2019-01-31T21:00:00.000000000
start_date: 2019-02-01T00:00:00.000000000 stop_date: 2019-02-28T21:00:00.000000000
start_date: 2019-03-01T00:00:00.000000000 stop_date: 2019-03-31T21:00:00.000000000
start_date: 2019-04-01T00:00:00.000000000 stop_date: 2019-04-30T21:00:00.000000000
start_date: 2019-05-01T00:00:00.000000000 stop_date: 2019-05-31T21:00:00.000000000
start_date: 2019-06-01T00:00:00.000000000 stop_date: 2019-06-30T21:00:00.000000000
start_date: 2019-07-01T00:00:00.000000000 stop_date: 2019-07-31T21:00:00.000000000
start_date: 2019-08-01T00:00:00.000000000 stop_date: 2019-08-31T21:00:00.000000000
start_date: 2019-09-01T00:00:00.000000000 stop_date: 2019-09-30T21:00:00.000000000
start_date: 2019-10-01T00:00:00.000000000 stop_date: 2019-10-31T21:00:00.000000000
start_date: 2019-11-01T00:00:00.000000000 stop_date: 2019-11-30T21:00:00.000000000
start_date: 2019-12-01T00:00:00.000000000 stop_date: 2019-12-31T21:00:00.000000000
star

Another possibility is to use directly the `stream_data` method of the Reader class on a given xarray.Dataset, for example on a subset of the input data:

In [14]:
reader = Reader(model='MSWEP',exp='past',source='3hourly')
data = reader.retrieve()
new_data = reader.stream(data["mtpr"], aggregation='3D')
new_data

<xarray.DataArray 'mtpr' (time: 21, lat: 1800, lon: 3600)>
dask.array<getitem, shape=(21, 1800, 3600), dtype=float32, chunksize=(21, 113, 225), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 1979-01-03T21:00:00
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
Attributes:
    units:        kg m**-2 s**-1
    history:      \n2024-02-15 17:43:58 AQUA💧: Variable renamed mtpr by fixer...
    paramId:      235055
    long_name:    Mean total precipitation rate
    cfVarName:    mtpr
    shortName:    mtpr
    factor:       9.25925925925926e-05
    offset:       0
    src_units:    mm 3h-1
    units_fixed:  1